In [1]:
# This is an initial device test run to see if the user feels all the nodes or if they need adjustmet

import serial 
import csv
import time
import struct


testmatrix = []
num = 0
# with open('C:/Users/santiago.arconada/Downloads/sensogram.txt') as csvfile:
with open('sensogram.txt') as csvfile:
    inputfile = csv.reader(csvfile)
    for row in inputfile:
        testmatrix.append(row)
        for i in range(len(row)):
            testmatrix[num][i] = int(row[i])
        num = num+1    
        # At this point we get all values in packets of data being integer, good job, had to loop through each array
        # as it comes in to convert it from a string to an integer.

# Might have to change the tty.usbserial to Mac specific address
ser = serial.Serial('/dev/cu.usbmodem1411',9600)
# ser = serial.Serial('/dev/cu.usbmodem53',9600)

node_matrix = []

# This section is for parsing the data inside the testmatrix
activ_array_new = []
# Counter = [13,50,5,50,7,50,8,50,9,50,13,50,10,50,11,50,6,50,13,50]
# Test run for all the nodes in a sequence
strength = 100
Counter = [13,200,3,strength,13,200,4,strength,13,200,5,strength,13,200,6,strength,13,200,7,strength,13,200,8,strength,13,200,9,strength,13,200,10,strength,13,200,11,strength,13,200,12,strength,13,200,13,strength]
b = 0

while b<1:

    line = ser.readline()
    print(line)
    if (ser.readline() == b'ready\r\n'):
        ser.write(b'1') # Need to include the b so that data can be sent
                        # this is python 3 syntax
#         ser.write(b'1') # This will be the initiate Serial listening command
            
        ser.write(struct.pack('>B', len(Counter))) # Something is working here, but I am not sure what it is haha
        print(struct.pack('>B', len(Counter)))
        c = 0
#         for y in range(len(Counter)//2):
        for y in range(21):
            ser.write(struct.pack('>B', Counter[c]))
            ser.write(struct.pack('>B', Counter[c+1]))
            print('Im sending %d %d' % (Counter[c], Counter[c+1]))
            c = c+2
#         ser.write(b Counter '\r') # This defines how many to wait for
        time.sleep(0.2)
    else:
        time.sleep(0.2)
            
    # So this part of the code is what defines the reading speed of the 'control board'
    # Each 100ms it checks the serial to see if there are any responses from the "display board"
#   # This way I can send values only when the Arduino is ready.
# ser.close()
    b = b+1

b'ready\r\n'
b','
Im sending 13 200
Im sending 3 100
Im sending 13 200
Im sending 4 100
Im sending 13 200
Im sending 5 100
Im sending 13 200
Im sending 6 100
Im sending 13 200
Im sending 7 100
Im sending 13 200
Im sending 8 100
Im sending 13 200
Im sending 9 100
Im sending 13 200
Im sending 10 100
Im sending 13 200
Im sending 11 100
Im sending 13 200
Im sending 12 100
Im sending 13 200


In [ ]:
Counter = [9,40,1,20,5,60]
print(Counter[0])
print(struct.pack('>B', 12))
for i in range(3):
    print(i)

In [ ]:
activ_array_new = []
for i in range(1):
    activ_array = []
    for q in range(7):
        if (testmatrix[i][testmatrix[i][q+6]] == 1):
            activ_array.append(testmatrix[i][q+6])
            activ_array.append(testmatrix[i][q+13])
    activ_array_new.append(activ_array) # We could send this as a packet of data to the Arduino (node,delay)
    print(activ_array)
    print(len(activ_array))

    Counter = len(activ_array) # How many values am I sending
    print('Counter is', Counter)
#     ser.write(Counter)
    n = 0
    for j in range(Counter//2):
        print('Node %d and delay %d', activ_array_new[i][n], activ_array_new[i][n+1]) # Substitute this by sending value on Serial
        #ser.write(activ_array_new[i][j] 0D 0A)
        n = n+2

In [ ]:
# print(testmatrix)

In [ ]:
# Code for the Arduino

void setup() {
pinMode(3,OUTPUT);
# And all others
Serial.begin(9600);
}

void loop() {
    if (Serial.available()) {
        while (# Command to start communication isn't there yet) {
            # Just wait
            }
            
        values = #read incoming data with count of how many im doing
        for (i=0; i<values,i++) {
            node_array[i] = #serialdata;
            delay_array[i] = #serialdata;
        }
        
        # Maybe I could combine these two for loops together
        
        # for loop reading x values of the serial
        for (i=0; i<values,i++) {
            digitalWrite(node_array[i],HIGH);
            # Maybe put a print to test out if it works
            delay(delay_array[i]);
            digitalWrite(node_array[i],LOW);
        }
        Serial.write('done'); # To tell python that we are done with the sequence
    }
}



In [ ]:
# This code is to be run as the final version, but I don't want to modify the code that works already above
# so I will use this as my testing platform

import serial 
import csv
import time
import struct
from PIL import Image
import random

%run _Functions.ipynb

testmatrix = LoadSensogram()

picmatrix = LoadImages('Picorder.txt')

# Connect to Arduino
ser = serial.Serial('/dev/cu.usbmodem1421',9600)

sampledown = 0
sampleup = 6

# # Define order array, need to change this up for the second round, i.e range(10,20)
order = list(range(sampledown,sampleup))
# Randomize the order
random.shuffle(order)

# This establishes serial communication and data sending
for i in order: # Loop through 20 images for the experiment
    TestPattern(ser,testmatrix,i)
    img = Image.open(','.join(picmatrix[i]))
    img.show()
    time.sleep(2)


In [ ]:
# This is the testing code, used to run through the patterns but not displaying the associated
# image, that the user will select among 5 presented.

import serial 
import random

%run _Functions.ipynb

testmatrix = LoadSensogram()

picmatrix = LoadImages('Picorder.txt')

# Connect to Arduino
ser = serial.Serial('/dev/cu.usbmodem1421',9600)
# ser = 'hi'

# Define the results matrix
results = []

# Re-randomize for testing round, need to change this up for the second round, i.e range(10,20)
order = list(range(sampledown,sampleup))
# Randomize the order
random.shuffle(order)

# This establishes serial communication and data sending
for i in order:
    
    TestPattern(ser,testmatrix,i)
    # MIGHT HAVE TO REDUCE THE DELAY TIME, IT IS TOO MUCH, OR MAYBE MOVE THE TEST FUNCTION UP
    time.sleep(1.5)    
    #After the data has been sent to the sensogram display the pictures and test the subject
    TestFunction(i,picmatrix,order)
    time.sleep(0.5)

In [ ]:
PrintScore(results)
DetailedInfo(results,order,1)